In [1]:
import pandas as pd
from prophet import Prophet
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare Data

In [2]:
def preparedata(file):
    cloud_cover = pd.read_csv(file, skipinitialspace = True)
    cloud_cover['Date'] = cloud_cover[['Year', 'Month', 'Day']].apply(lambda x: "{0}-{1}-{2}".format(str(x[0]).zfill(2), str(x[1]).zfill(2), str(x[2]).zfill(2)), axis=1) 
    cloud_cover['Time'] = cloud_cover[['Hour', 'Minute']].apply(lambda x: "{}:{}".format(str(x[0]).zfill(2), str(x[1]).zfill(2)), axis=1)
    cloud_cover.drop(["Year", "Month", "Day", "Hour", "Minute", "Second"], axis=1, inplace=True)  
    cloud_cover = cloud_cover.rename(columns={"CloudCover":"cloud_cover"})
    cloud_cover = cloud_cover.groupby(['Date', 'Time']).agg(cloud_cover=('cloud_cover','mean')).reset_index()
    cloud_cover['DateTime'] = cloud_cover[['Date', 'Time']].apply(lambda x: "{} {}".format(str(x[0]).zfill(2), str(x[1]).zfill(2)), axis=1)
    cloud_cover.drop(["Date", "Time"], axis=1, inplace=True)        
    cloud_cover['DateTime'] = pd.to_datetime(cloud_cover['DateTime'], format="%Y-%m-%d %H:%M")
    cloud_cover.dropna(inplace=True)
    return cloud_cover

In [3]:
cloud_cover = preparedata('Bangkhuntean_CloudCover_2021-16Nov-16Dec.csv')
cloud_cover

,cloud_cover,DateTime
0,0.990000,2021-11-16 11:15:00
1,0.989000,2021-11-16 11:16:00
2,0.987667,2021-11-16 11:17:00
3,0.987000,2021-11-16 11:18:00
4,0.987000,2021-11-16 11:19:00
...,...,...
15056,0.168000,2021-12-16 13:58:00
15057,0.169000,2021-12-16 13:59:00
15058,0.147000,2021-12-16 14:00:00
15059,0.145000,2021-12-16 14:01:00


In [4]:
df = cloud_cover[['DateTime', 'cloud_cover']]
df.columns = ['ds', 'y']
df

,ds,y
0,2021-11-16 11:15:00,0.990000
1,2021-11-16 11:16:00,0.989000
2,2021-11-16 11:17:00,0.987667
3,2021-11-16 11:18:00,0.987000
4,2021-11-16 11:19:00,0.987000
...,...,...
15056,2021-12-16 13:58:00,0.168000
15057,2021-12-16 13:59:00,0.169000
15058,2021-12-16 14:00:00,0.147000
15059,2021-12-16 14:01:00,0.145000


# Train/Test dataset

In [8]:
#testset one day
train = df[df['ds'] < '2021-12-15 09:11:00']
test = df[df['ds'] >= '2021-12-15 09:11:00']

print('train Shape', train.shape)
print('test Shape', test.shape)

train Shape (14312, 2)
test Shape (749, 2)


# Model use testset one day

In [13]:
model = Prophet()
model.fit(train)
predict = model.predict(test)
predict

23:31:55 - cmdstanpy - INFO - Chain [1] start processing
23:32:26 - cmdstanpy - INFO - Chain [1] done processing


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,daily_lower,daily_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2021-12-15 09:11:00,0.208151,-0.074925,0.474418,0.208151,0.208151,-0.011073,-0.011073,-0.011073,0.224731,0.224731,0.224731,-0.235804,-0.235804,-0.235804,0.0,0.0,0.0,0.197078
1,2021-12-15 09:12:00,0.208170,-0.066252,0.461512,0.208170,0.208170,-0.010732,-0.010732,-0.010732,0.225151,0.225151,0.225151,-0.235883,-0.235883,-0.235883,0.0,0.0,0.0,0.197438
2,2021-12-15 09:13:00,0.208189,-0.041940,0.478342,0.208189,0.208189,-0.010376,-0.010376,-0.010376,0.225586,0.225586,0.225586,-0.235962,-0.235962,-0.235962,0.0,0.0,0.0,0.197813
3,2021-12-15 09:14:00,0.208207,-0.068655,0.463184,0.208207,0.208207,-0.010005,-0.010005,-0.010005,0.226035,0.226035,0.226035,-0.236041,-0.236041,-0.236041,0.0,0.0,0.0,0.198202
4,2021-12-15 09:15:00,0.208226,-0.060747,0.477868,0.208226,0.208226,-0.009621,-0.009621,-0.009621,0.226499,0.226499,0.226499,-0.236120,-0.236120,-0.236120,0.0,0.0,0.0,0.198605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,2021-12-16 13:58:00,0.240620,0.130254,0.762892,0.124065,0.386114,0.194372,0.194372,0.194372,0.264982,0.264982,0.264982,-0.070611,-0.070611,-0.070611,0.0,0.0,0.0,0.434992
745,2021-12-16 13:59:00,0.240639,0.121417,0.750869,0.123941,0.386443,0.193283,0.193283,0.193283,0.263711,0.263711,0.263711,-0.070428,-0.070428,-0.070428,0.0,0.0,0.0,0.433922
746,2021-12-16 14:00:00,0.240658,0.109947,0.759328,0.124278,0.386771,0.192188,0.192188,0.192188,0.262433,0.262433,0.262433,-0.070245,-0.070245,-0.070245,0.0,0.0,0.0,0.432846
747,2021-12-16 14:01:00,0.240676,0.128188,0.770808,0.124653,0.387099,0.191088,0.191088,0.191088,0.261150,0.261150,0.261150,-0.070062,-0.070062,-0.070062,0.0,0.0,0.0,0.431764


In [14]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ds                          749 non-null    datetime64[ns]
 1   trend                       749 non-null    float64       
 2   yhat_lower                  749 non-null    float64       
 3   yhat_upper                  749 non-null    float64       
 4   trend_lower                 749 non-null    float64       
 5   trend_upper                 749 non-null    float64       
 6   additive_terms              749 non-null    float64       
 7   additive_terms_lower        749 non-null    float64       
 8   additive_terms_upper        749 non-null    float64       
 9   daily                       749 non-null    float64       
 10  daily_lower                 749 non-null    float64       
 11  daily_upper                 749 non-null    float64       

In [10]:
predict1 = predict[['ds', 'yhat']]
predict1

,ds,yhat
0,2021-12-15 09:11:00,0.197078
1,2021-12-15 09:12:00,0.197438
2,2021-12-15 09:13:00,0.197813
3,2021-12-15 09:14:00,0.198202
4,2021-12-15 09:15:00,0.198605
...,...,...
744,2021-12-16 13:58:00,0.434992
745,2021-12-16 13:59:00,0.433922
746,2021-12-16 14:00:00,0.432846
747,2021-12-16 14:01:00,0.431764


# forecasting with 30 minutes by dataset 30 minutes ago

In [ ]:
df_30m = df.iloc[len(df)-30:]
df_30m

In [ ]:
cloud_cover.set_index('DateTime', 'cloud_cover', inplace=True)
index_30_min = pd.date_range(cloud_cover.index[-1], freq='1T', periods = 31) 
df_index_30_min = pd.DataFrame(index_30_min)
df_index_30_min.columns = ['ds']
df_index_30_min

In [ ]:
model_30m = Prophet()
model_30m.fit(df_30m)
fcast30m = model_30m.predict(df_index_30_min)
fcast30min = fcast30m[['ds', 'yhat_lower', 'yhat_upper', 'yhat']]
fcast30min

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='lines', x= X_train['ds'], y=X_train["y"], name='Train'))
fig.add_trace(go.Scatter(mode='lines', x=fcast1['ds'], y = fcast1["yhat"], name='Forecast prophet predict test set'))
fig.add_trace(go.Scatter(mode='lines', x=X_test['ds'], y = X_test["y"], name='Test'))
fig.add_trace(go.Scatter(mode='lines', x=df['ds'], y = df["y"],  name='Real Data'))
fig.add_trace(go.Scatter(mode='lines', x=fcast30min['ds'], y =fcast30min["yhat"],  name='Forecast prophet predict future 30 min'))

fig.update_layout(
    autosize=True,
    height=600,
    title="Prophet Model",
    xaxis_title="Date",
    yaxis_title="Cloud cover",
)


fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1.zoom x1",
                     step="hour",
                     stepmode="backward"),
                 dict(count=2,
                     label="2.zoom x2",
                     step="hour",
                     stepmode="backward"),
                dict(count=3,
                     label="3.zoom x3",
                     step="day",
                     stepmode="backward"),              

                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

# Evaluating forecast accuracy

- **MAE** ย่อมาจาก Mean Absolute Error หรือเรียกอีกชื่อหนึ่งว่า L1 Loss ค่า MAE นี้ชื่อก็บอกอยู่แล้วว่าใช้ Absolute มาช่วยทำให้ค่า Error กลายเป็นบวก สูตรคำนวนจึงเป็นการนำค่า Error มาใส่ Absolute ก่อนที่จะนำมาหาค่าเฉลี่ยของ Error
- **MSE** ย่อมาจาก Mean Square Error หรือเรียกอีกชื่อหนึ่งว่า L2 Loss เช่นเดียวกัน ค่า MSE จะมีการทำให้ค่า Error กลายเป็นบวกก่อนโดยการนำค่า Error มายกกำลังสอง ก่อนที่จะนำค่า Error มาหาค่าเฉลี่ย
- **RMSE** ย่อมาจาก Root Mean Square Error เป็น Loss Function ที่จะนำค่า MSE มาใส่ Square Root จึงทำให้มีคุณสมบัติที่คล้ายกับค่า MSE แต่ต่างกันตรงที่ หน่วยของค่า Error จะไม่มีเลขยกกำลังสอง จึงทำให้อ่านค่าได้ง่ายกว่า เนื่องจากหน่วยของ RMSE นั้นมีหน่วยเดียวกันกับค่าที่โมเดลทำนายไว้

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
prophet_mse_error = mean_squared_error(X_test["y"], fcast1["yhat"], squared=True)
prophet_rmse_error = mean_squared_error(X_test["y"], fcast1["yhat"], squared=False)
prophet_mae_error = mean_absolute_error(X_test["y"], fcast1["yhat"])
prophet_r2 = r2_score(X_test["y"], fcast1["yhat"])

print(f'MSE Error: {prophet_mse_error}\nRMSE Error: {prophet_rmse_error}\nMAE: {prophet_mae_error}\nfr2_score: {prophet_r2}')

----------------------------------------------------------------------------------------------

# Use cross validation

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
cv = TimeSeriesSplit(n_splits=5)
i = 0
for train,test in cv.split(df):
    print('TRAIN:', train, 'TEST:', test) 
    i=i+1
    print ("No of observations under train%s=%s" % (i, len(train)))
    print ("No of observations under test%s=%s" % (i, len(test)))

In [ ]:
train1, test1 = df.iloc[:2511], df.iloc[2511:5021]
train2, test2 = df.iloc[:5021], df.iloc[5021:7531]
train3, test3 = df.iloc[:7531], df.iloc[7531:10041]
train4, test4 = df.iloc[:10041], df.iloc[10041:12551]
train5, test5 = df.iloc[:12551], df.iloc[12551:15061]

In [ ]:
model1 = Prophet()
model1.fit(train1)
pred1 = model1.predict(test1)
pred1

In [ ]:
model2 = Prophet()
model2.fit(train2)
pred2 = model2.predict(test2)
pred2

In [ ]:
model3 = Prophet()
model3.fit(train3)
pred3 = model3.predict(test3)
pred3

In [ ]:
model4 = Prophet()
model4.fit(train4)
pred4 = model4.predict(test4)
pred4

In [ ]:
model5 = Prophet()
model5.fit(train5)
pred5 = model5.predict(test5)
pred5

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(mode='lines', x= train1['ds'], y=train1["y"], name='Train 1'))
fig.add_trace(go.Scatter(mode='lines', x= test1['ds'], y=test1["y"], name='Test 1'))
fig.add_trace(go.Scatter(mode='lines', x= pred1['ds'], y= pred1["yhat"], name='Prediction 1'))

fig.add_trace(go.Scatter(mode='lines', x= train2['ds'], y= train2["y"],  name='Train 2'))
fig.add_trace(go.Scatter(mode='lines', x= test2['ds'], y=test2["y"],  name='Test 2'))
fig.add_trace(go.Scatter(mode='lines', x= pred2['ds'], y= pred2["yhat"], name='Prediction 2'))

fig.add_trace(go.Scatter(mode='lines', x= train3['ds'], y= train3["y"],  name='Train 3'))
fig.add_trace(go.Scatter(mode='lines', x= test3['ds'], y=test3["y"],  name='Test 3'))
fig.add_trace(go.Scatter(mode='lines', x= pred3['ds'], y= pred3["yhat"], name='Prediction 3'))

fig.add_trace(go.Scatter(mode='lines', x= train4['ds'], y= train4["y"],  name='Train 4'))
fig.add_trace(go.Scatter(mode='lines', x= test4['ds'], y=test4["y"],  name='Test 4'))
fig.add_trace(go.Scatter(mode='lines', x= pred4['ds'], y= pred4["yhat"], name='Prediction 4'))

fig.add_trace(go.Scatter(mode='lines', x= train5['ds'], y= train5["y"],  name='Train 5'))
fig.add_trace(go.Scatter(mode='lines', x= test5['ds'], y=test5["y"],  name='Test 5'))
fig.add_trace(go.Scatter(mode='lines', x= pred5['ds'], y= pred5["yhat"], name='Prediction 5'))

fig.update_layout(
    autosize=True,
    height=600,
    title="Prophet Model",
    xaxis_title="Date",
    yaxis_title="Cloud cover",
)


fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1.zoom x1",
                     step="hour",
                     stepmode="backward"),
                 dict(count=2,
                     label="2.zoom x2",
                     step="hour",
                     stepmode="backward"),
                dict(count=3,
                     label="3.zoom x3",
                     step="day",
                     stepmode="backward"),              

                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

fig.show()

In [ ]:
prophet_rmse_error1 = mean_squared_error(test1["y"], pred1["yhat"], squared=False)
prophet_rmse_error2 = mean_squared_error(test2["y"], pred2["yhat"], squared=False)
prophet_rmse_error3 = mean_squared_error(test3["y"], pred3["yhat"], squared=False)
prophet_rmse_error4 = mean_squared_error(test4["y"], pred4["yhat"], squared=False)
prophet_rmse_error5 = mean_squared_error(test5["y"], pred5["yhat"], squared=False)

print ("RMSE1 : ", prophet_rmse_error1)
print ("RMSE2 : ", prophet_rmse_error2)
print ("RMSE3 : ", prophet_rmse_error3)
print ("RMSE4 : ", prophet_rmse_error4)
print ("RMSE5 : ", prophet_rmse_error5)

Overall_RMSE = (prophet_rmse_error1+prophet_rmse_error2+prophet_rmse_error3+prophet_rmse_error4+prophet_rmse_error5)/5
print ("Overall RMSE:", Overall_RMSE) 

---

In [ ]:
cloud_cover1 = pd.read_csv('Bangkhuntean_CloudCover_2021-16Nov-16Dec.csv',skipinitialspace = True)
cloud_cover1['Date'] = cloud_cover1[['Year', 'Month', 'Day']].apply(lambda x: "{0}-{1}-{2}".format(str(x[0]).zfill(2), str(x[1]).zfill(2), str(x[2]).zfill(2)), axis=1) 
cloud_cover1.drop(["Year", "Month", "Day", "Hour", "Minute", "Second"], axis=1, inplace=True)  
cloud_cover1 = cloud_cover1.rename(columns={"CloudCover":"cloud_cover"})
cloud_cover1 = cloud_cover1.groupby(['Date']).agg(cloud_cover=('cloud_cover','mean')).reset_index()    
cloud_cover1['Date'] = pd.to_datetime(cloud_cover1['Date'], format="%Y-%m-%d")
cloud_cover1.dropna(inplace=True)
cloud_cover1

In [ ]:
df1 = cloud_cover1[['Date', 'cloud_cover']]
df1.columns = ['ds', 'y']
df1

In [ ]:
from sklearn.model_selection import train_test_split
train6,test6 = train_test_split(df1, train_size=0.8, shuffle=False)
print('Train Shape', train6.shape)
print('Test Shape', test6.shape)

In [ ]:
model2 = Prophet()
model2.fit(train6)
fcast4 = model2.predict(test6)
fcast5 = fcast4[['ds','yhat']]
fcast5

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
plt.xlabel('Date')
plt.ylabel('cloud_cover')
sns.lineplot(x= train6['ds'], y=train6["y"],label = 'Train') 
sns.lineplot(x=fcast5['ds'], y = fcast5["yhat"], label = 'forecast_prophet_predict')
sns.lineplot(x=test6['ds'], y = test6["y"], label = 'Test')

In [ ]:
prophet_mse_error1 = mean_squared_error(test6["y"], fcast5["yhat"], squared=True)
prophet_rmse_error1 = mean_squared_error(test6["y"], fcast5["yhat"], squared=False)
prophet_mae_error1 = mean_absolute_error(test6["y"], fcast5["yhat"])
prophet_r21 = r2_score(test6["y"], fcast5["yhat"])

print(f'MSE Error: {prophet_mse_error1}\nRMSE Error: {prophet_rmse_error1}\nMAE: {prophet_mae_error1}\nfr2_score: {prophet_r21}')